Pandas.DataFrame.ewm: 指数加权移动窗口函数详解

pandas.DataFrame.ewm 是 Pandas 库中一个非常强大的功能，用于计算指数加权移动（Exponentially Weighted Moving, EWM）统计量。与简单移动平均（Simple Moving Average）给予窗口内所有数据点相同权重不同，EWM 会给予近期的数据点更高的权重，而较早的数据点权重则会呈指数级衰减。这使得 EWM 对于时间序列数据中的最新变化更为敏感，能够更好地反映近期趋势。

ewm 本身并不直接进行计算，而是在 DataFrame 或 Series 上创建一个 ExponentialMovingWindow 对象。然后，您可以链接各种聚合函数（如 .mean(), .std(), .var() 等）来计算实际的指数加权统计值。

核心概念：权重衰减
EWM 的核心思想是权重。每个数据点的权重由一个称为 平滑因子 (alpha, α) 的参数决定。一个时间点 t 的指数加权移动平均（EWMA）的计算方式可以递归地表示为：

$$
EWMA_t = (1 − alpha) \cdot (EWMA_{t − 1}) + alpha \cdot x_t
$$

其中：

x_t 是在时间点 t 的观测值。

EWMA_t−1 是在前一个时间点的指数加权移动平均。

alpha 是平滑因子，其值在 0 和 1 之间。
alpha 越大，对近期观测值的权重就越大，从而使得平均值对新数据的反应更迅速。

关键参数:

- com (Center of Mass, 重心): 通过重心来指定衰减。

    alpha=1 / (1 + com)

    com 值越大，alpha 越小，衰减越慢。

- span (Span, 跨度): 通过跨度来指定衰减。

    alpha=2 / (span + 1)

    这通常用于描述“N-周期 EWMA”。例如，金融图表中常见的 20 日 EMA 对应 span=20。span 越大，

    alpha 越小，曲线越平滑。

- halflife (Half-life, 半衰期): 通过半衰期来指定衰减，即权重衰减到其一半所需的时间周期。

    alpha = 1− exp(log(0.5) / halflife)

    halflife 越大，alpha 越小。

- alpha (Alpha, 平滑因子): 直接指定平滑因子alpha。

在一次调用中，您只能指定 com, span, halflife, alpha 中的一个。

其他常用参数包括：

min_periods (int, 默认为 0): 窗口内具有值的最小观测点数量，否则结果为 NaN。

adjust (bool, 默认为 True): 是否在初创期进行调整，以解决权重不平衡问题。当 adjust=True 时，EWMA 的计算会除以一个衰减的调整因子，这在处理较短的时间序列时更为精确。当 adjust=False 时，使用递归公式进行计算。

ignore_na (bool, 默认为 False): 是否在计算权重时忽略缺失值。

常用聚合函数
在使用 .ewm() 之后，您可以调用以下聚合函数：

.mean(): 计算指数加权移动平均值 (EWMA)。

.std(): 计算指数加权移动标准差 (EWMSD)。

.var(): 计算指数加权移动方差 (EWMV)。

.corr(): 计算成对的指数加权移动相关系数。

.cov(): 计算成对的指数加权移动协方差。

In [91]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [92]:
df = pd.read_csv('ewm.csv')
df

,y
0,0.000000
1,0.064261
2,-0.004961
3,-0.045155
4,-0.040670
...,...
301,-0.073853
302,-0.083527
303,-0.075722
304,-0.107238


In [93]:
df.describe()

,y
count,306.000000
mean,-0.033493
std,0.072499
min,-0.197629
25%,-0.083579
50%,-0.043971
75%,0.022068
max,0.141682


In [94]:
fig = go.Figure()
x: np.ndarray = np.arange(len(df))

y = df["y"]
# 添加3组不同的数据
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        name="original",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.update_layout(height=600, title="Pixel Move")

fig.show()

In [95]:
# 计算指数加权移动标准差
df["alpha_0.3"] = df["y"].ewm(alpha=0.3, adjust=False).mean()

# 计算 5 日指数加权移动平均 (EMA)
df["ema_5"] = df["y"].ewm(span=5, adjust=False).mean()

# 计算重心为 0.5 的指数加权移动平均
df["com_0.5"] = df["y"].ewm(com=0.5, adjust=False).mean()

# 计算指数加权移动标准差
df["halflife_5"] = df["y"].ewm(halflife=5, adjust=False).mean()

In [96]:
def moment(old: float, new: float, beta: float = 0.7):
    return beta * old + (1 - beta) * new

In [97]:
def use_moment():
    x = df["y"]
    y = [x[0]]
    for i in range(1, len(x)):
        y.append(moment(y[-1], x[i]))
    print(len(y))
    df["beta_0.7"] = y
use_moment()

306


In [98]:
fig = go.Figure()
x: np.ndarray = np.arange(len(df))

fig.add_trace(
    go.Scatter(
        x=x,
        y=df["y"],
        name="original",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df["alpha_0.3"],
        name="alpha_0.3",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df["ema_5"],
        name="ema_5",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df["com_0.5"],
        name="com_0.5",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df["halflife_5"],
        name="halflife_5",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df["beta_0.7"],
        name="beta_0.7",  # 名字
        mode="lines",  # ['lines', 'markers', 'text'] joined with '+' characters
        hoverinfo="y",  # 鼠标悬停显示的数据
    )
)

fig.update_layout(height=600, title="ewm compare")

fig.show()